Todoist

https://developer.todoist.com/rest/v1/#update-a-task

install 
pip install todoist-api-python

must be python 3.8 or higher


In [149]:
import os
import numpy as np
import pandas as pd
from todoist_api_python.api import TodoistAPI
import json
import datetime


In [4]:
# read token
fname = 'todoist_api_token.txt'
with open (fname) as f:
    todoist_api = f.readlines()
todoist_api = todoist_api[0]

In [ ]:
api = TodoistAPI(todoist_api)

try:
    projects = api.get_projects()
    print(projects)
except Exception as error:
    print(error)

In [7]:
# test adding tasks
api = TodoistAPI(todoist_api)
try:
    task = api.add_task(
        content='Test adding task from python',
        due_string='today',
        due_lang='en',
        priority=1,
    )
    print(task)
except Exception as error:
    print(error)

Task(comment_count=0, completed=False, content='Test adding task from python', created='2022-07-24T00:16:27.480301Z', creator=25159017, description='', id=6029698491, project_id=2222925285, section_id=0, priority=1, url='https://todoist.com/showTask?id=6029698491', assignee=None, assigner=0, due=Due(date='2022-07-23', recurring=False, string='today', datetime=None, timezone=None), label_ids=[], order=4, parent_id=None, sync_id=None)


In [36]:
# get project id list
api = TodoistAPI(todoist_api)
try:
    projects = api.get_projects()
except Exception as error:
    print(error)
# project id into dictionary
projects_dict = {}
for i in range(len(projects)):
    projects_dict[projects[i].name] = projects[i].id


Here can add enable google api
https://developers.google.com/sheets/api/quickstart/python


In [38]:
# find todoist tasks with recurrent date and time

# get task
api = TodoistAPI(todoist_api)
try:
    tasks = api.get_tasks()
except Exception as error:
    print(error)

In [59]:

# project id into dictionary
recurrent_task_list = []
recurrent_task_datetime = []
for i in range(len(tasks)):
    if tasks[i].due:
        if tasks[i].due.recurring:
            recurrent_task_list.append(tasks[i].id)

len(recurrent_task_list)

345

label_ids
* Array of Integers
* IDs of labels associated with the task.

In [154]:
# creat label dictionary
def get_label_dict(todoist_api):
    api = TodoistAPI(todoist_api)
    try:
        labels = api.get_labels()
    except Exception as error:
        print(error)
    # label into dictionary
    label_dict = {}
    for i in range(len(labels)):
        label_dict[labels[i].name] = labels[i].id
    return label_dict

label_dict = get_label_dict(todoist_api)
label_dict

{'Prepared': 2159777797,
 'Planned': 2156248492,
 'Blocker': 2160067604,
 'Reminder.NotNeeded': 2159836237,
 'Reminded': 2157873695,
 'Reminded.3days': 2159303616,
 '_Deadline': 2157316483,
 '_Deadline.PastDue': 2155612158,
 '_Meeting.Organizer': 2157518475,
 '_Meeting.Events': 2160586699,
 '_Meeting.Presenter': 2159659217,
 '_Meeting.Discuss': 2159659307,
 '_Meeting.Attend': 2159659143,
 '_Meeting.Seminar': 2156481252,
 '_Meeting.Course': 2159863553,
 '_Computer': 2153733639,
 '_Computer.Log.BetterWorks': 2161136075,
 '_JaaS': 2160330221,
 '_Confluence': 2160412501,
 '_Office': 2160771751,
 '_Device': 2153733636,
 '_Read': 2156807637,
 '_Watch': 2157231717,
 '_Listen': 2159296825,
 '_Home': 2153733640,
 '_Project': 2153734096,
 '_Goal.Sprint': 2157397327,
 '_Goal.Project': 2160758326,
 '_Goal.Banked': 2159293296,
 '_Agenda': 2155914986,
 '_Agenda.RobBaron': 2159631330,
 '_Agenda.Daria': 2160079387,
 '_Agenda.TedLao': 2160249522,
 '_Agenda.Ariel': 2160260290,
 '_Agenda.Saman': 21602602

Get recurrent tasks

In [82]:
# create dataframe
df = pd.DataFrame()

# get task id first
a = []
for i in range(len(tasks)):
    a.append(tasks[i].id)
df['id'] = a

# get has due date or not
a = np.zeros((len(tasks),1), dtype=bool)
for i in range(len(tasks)):
    if tasks[i].due:
        a[i] = True
df['due_exist'] = a

# get recurrent (r), string (b), datetime (c)
r = np.zeros((len(tasks),1), dtype=bool)
b = np.empty((len(tasks),1), dtype=object)
c = b
for i in range(len(tasks)):
    if tasks[i].due:       
        if tasks[i].due.recurring:
            r[i] = True
        if tasks[i].due.string:
            b[i] = tasks[i].due.string
        if tasks[i].due.datetime:
            c[i] = tasks[i].due.datetime
df['recurrent'] = r
df['string'] = b
df['datetime'] = c

# 

,id,due_exist,recurrent,string,datetime
0,3557984409,True,True,every day,every day
1,3557989790,False,False,None,None
2,3558000864,True,True,every day,every day
3,3559544883,False,False,None,None
4,3559544907,False,False,None,None
...,...,...,...,...,...
1645,6029649144,True,False,Jul 24,Jul 24
1646,6029651357,True,False,Jul 29,Jul 29
1647,6029661843,True,False,Jul 27,Jul 27
1648,6029697691,True,False,today,today


# let's see how to upload csv sheet with more customed task list

In [97]:
tasks[0]

Task(comment_count=0, completed=False, content='\U0001faa5Brush teeth', created='2019-12-11T07:08:06.000000Z', creator=25159017, description='', id=3557984409, project_id=2230726015, section_id=35780658, priority=1, url='https://todoist.com/showTask?id=3557984409', assignee=None, assigner=0, due=Due(date='2022-07-24', recurring=True, string='every day', datetime=None, timezone=None), label_ids=[2153733640], order=4, parent_id=None, sync_id=None)

In [99]:
fname = 'todoist-python.csv'
dfcsv = pd.read_csv(fname)

In [100]:
dfcsv

,content,label,duration,time,priority,order,recurrent_string,start_week,days_minus_parent,date
0,Morning Routine,_Home,60,6:00,4,1,weekday,1,NaN,25-Jul
1,Weekend Beauty Routine,_Home,120,6:00,4,1,weekend,1,NaN,30-Jul
2,GTD,_Computer,60,7:00,4,1,weekday,1,NaN,25-Jul
3,GTD Weekend,_Computer,180,8:00,4,1,weekend,1,NaN,30-Jul
4,GTD sun,_Computer,60,8:00,4,1,weekly,1,NaN,31-Jul
...,...,...,...,...,...,...,...,...,...,...
81,Prep Yoga,_Office,10,11:50,4,2,NaN,1,0.0,27-Jul
82,Airsoft,_Out @Planned,150,19:00,3,1,Sat,1,NaN,30-Jul
83,Go to Airsoft,_Out.Travel,30,18:30,3,2,NaN,1,0.0,30-Jul
84,Prep to go to Airsoft,_Home,30,18:00,3,2,NaN,1,0.0,30-Jul


Pandas frequency
https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-offset-aliases

pd.to_period
https://pandas.pydata.org/docs/reference/api/pandas.DatetimeIndex.to_period.html#pandas.DatetimeIndex.to_period

pd.DatetimeIndex
https://pandas.pydata.org/docs/reference/api/pandas.DatetimeIndex.html

In [101]:
pd.Timestamp.today()

Timestamp('2022-07-23 19:23:38.734076')

In [102]:
pd.to_datetime(['2022-07-26'])

DatetimeIndex(['2022-07-26'], dtype='datetime64[ns]', freq=None)

In [129]:
startdate = '2022-08-22'
enddate = pd.to_datetime(startdate) + pd.DateOffset(7*4)
freqcode = '2B'
offsetday = 1
pd.date_range(startdate, enddate, freq=freqcode)


DatetimeIndex(['2022-08-22', '2022-08-24', '2022-08-26', '2022-08-30',
               '2022-09-01', '2022-09-05', '2022-09-07', '2022-09-09',
               '2022-09-13', '2022-09-15', '2022-09-19'],
              dtype='datetime64[ns]', freq='2B')

In [111]:
# example how to add off set day
pd.to_datetime(startdate) + pd.DateOffset(offsetday)

Timestamp('2022-08-02 00:00:00')

In [130]:
# every 2nd Thu
freqcode = '2W-THU'
offsetday = 0
pd.date_range((pd.to_datetime(startdate) + pd.DateOffset(offsetday)), enddate, freq=freqcode)

DatetimeIndex(['2022-08-25', '2022-09-08'], dtype='datetime64[ns]', freq='2W-THU')

In [148]:
# third thu of the month
startdate_string = '2022-08-22'
startdate = pd.to_datetime(startdate_string) 
enddate = startdate + pd.DateOffset(7*4)
freqcode = 'W-THU'
offsetday = 0
weeknum = 3

# get first month's third thu and next month's third thu and give the one that's after start date and before end date
thedate = []
if weeknum > 0: # if there is a week num requirement
    # get the first month's third thu
    d1 = pd.to_datetime(startdate) + pd.offsets.MonthBegin(n=-1)
    datelist = pd.date_range(d1, enddate, freq=freqcode)
    datelist = datelist[weeknum-1] # 2 because the first value index is 0
    # check if is after start date
    if datelist >= startdate:
        thedate = datelist
    else:
        # get the second month's third thu
        d2 = pd.to_datetime(startdate) + pd.offsets.MonthBegin(n=1)
        datelist = pd.date_range(d2, enddate, freq=freqcode)
        datelist = datelist[weeknum-1] # 2 because the first value index is 0
        if datelist <= enddate:
            thedate = datelist
        else:
            thedate = []
            raise Exception('no date within the specified range')
    print(thedate)



2022-09-15 00:00:00


due_datetime

RC3339 format
https://medium.com/easyread/understanding-about-rfc-3339-for-datetime-formatting-in-software-engineering-940aa5d5f68a

In [ ]:
# convert time
datetime.datetime.now()

datelist[0].isoformat('T')